<a href="https://colab.research.google.com/github/Sun-of-a-beach/British_Airways_Virtual_Internship/blob/main/%20%20%20%20British_Airways_Virtual_Internship/Task_1_Web_Scrapping_%26_Data_Analysis%20%20/British_Airways_Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Loading Libraries**

In [ ]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime
# Data Manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

import re
from collections import Counter
#beautifulsoup
import requests
from bs4 import BeautifulSoup

##**Web Scrapping**

In [ ]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
result = requests.get(base_url)
result.raise_for_status()
doc = BeautifulSoup(result.text, "html.parser")
print(doc.prettify())

<!DOCTYPE doctype html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 9]>    <html class="no-js lt-ie10" lang="en-GB"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-GB">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <title>
   British Airways Customer Reviews - SKYTRAX
  </title>
  <!-- Google Chrome Frame for IE -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- mobile meta -->
  <meta content="True" name="HandheldFriendly"/>
  <meta content="320" name="MobileOptimized"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
   <!-- icons & favicons -->
   <link href="https://www.airlinequality.com/wp-content/themes/airlinequality2014new/library/image

In [ ]:
pages = 35
page_size = 100

data = []
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parsed content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    reviews = parsed_content.select('article[itemprop="review"]')
    for review in reviews:
        date = review.find('time', {'itemprop': 'datePublished'})['datetime']
        review_id = review.find('div', {'class': 'body'})['id'][6:]
        rating_element = review.find('div', {'class': 'rating-10'}).find('span', {'itemprop': 'ratingValue'})
        if rating_element is not None:
            rating = rating_element.get_text().strip()
        else:
            rating = 'na'

        ratings_table = review.select_one('table.review-ratings')
        rows = ratings_table.select('tr')
        review_dict = {'date': date, 'rating': rating, 'rev_id':review_id}
        ratings = {}
        for row in rows:
                header = row.find('td', class_='review-rating-header')
                value = row.find('td', class_='review-value')
                stars = row.find_all('span', class_='star fill')

                if header is not None and value is not None:
                    review_dict[header.text.strip()] = value.text.strip()

                if stars:
                    review_dict[header.text.strip()] = len(stars)


        comment = review.select_one('div[itemprop="reviewBody"]').get_text().strip()
        review_dict['comment'] = comment
        data.append(review_dict)

    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 100 total reviews
Scraping page 3
   ---> 100 total reviews
Scraping page 4
   ---> 100 total reviews
Scraping page 5
   ---> 100 total reviews
Scraping page 6
   ---> 100 total reviews
Scraping page 7
   ---> 100 total reviews
Scraping page 8
   ---> 100 total reviews
Scraping page 9
   ---> 100 total reviews
Scraping page 10
   ---> 100 total reviews
Scraping page 11
   ---> 100 total reviews
Scraping page 12
   ---> 100 total reviews
Scraping page 13
   ---> 100 total reviews
Scraping page 14
   ---> 100 total reviews
Scraping page 15
   ---> 100 total reviews
Scraping page 16
   ---> 100 total reviews
Scraping page 17
   ---> 100 total reviews
Scraping page 18
   ---> 100 total reviews
Scraping page 19
   ---> 100 total reviews
Scraping page 20
   ---> 100 total reviews
Scraping page 21
   ---> 100 total reviews
Scraping page 22
   ---> 100 total reviews
Scraping page 23
   ---> 100 total reviews
Scraping page 24
   

## **Creating a DataFrame to store everything.**

In [ ]:
df = pd.DataFrame(data)
print(df.head())

         date rating  rev_id Aircraft Type Of Traveller      Seat Type  \
0  2023-02-23      4  835501     A320    Couple Leisure  Economy Class   
1  2023-02-18      5  834045     A320    Family Leisure  Economy Class   
2  2023-02-18      1  833914     A320      Solo Leisure  Economy Class   
3  2023-02-16      8  833402      NaN    Couple Leisure  Economy Class   
4  2023-02-15      1  833131      NaN    Couple Leisure  Economy Class   

                 Route     Date Flown  Seat Comfort  Cabin Staff Service  \
0      Faro to Gatwick  February 2023           2.0                  2.0   
1       Oslo to London  February 2023           3.0                  3.0   
2     London to Madrid  February 2023           2.0                  1.0   
3  Islamabad to London  February 2023           4.0                  5.0   
4  Edinburgh to London  February 2023           3.0                  2.0   

   Food & Beverages  Ground Service  Value For Money Recommended  \
0               1.0           

In [ ]:
df

,date,rating,rev_id,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Value For Money,Recommended,comment,Inflight Entertainment,Wifi & Connectivity
0,2023-02-23,4,835501,A320,Couple Leisure,Economy Class,Faro to Gatwick,February 2023,2.0,2.0,1.0,5.0,2.0,no,✅ Trip Verified | Very competent check in sta...,NaN,NaN
1,2023-02-18,5,834045,A320,Family Leisure,Economy Class,Oslo to London,February 2023,3.0,3.0,1.0,2.0,3.0,no,"✅ Trip Verified | Check in was so slow, no se...",NaN,NaN
2,2023-02-18,1,833914,A320,Solo Leisure,Economy Class,London to Madrid,February 2023,2.0,1.0,1.0,1.0,2.0,no,✅ Trip Verified | My review relates to the ap...,NaN,NaN
3,2023-02-16,8,833402,NaN,Couple Leisure,Economy Class,Islamabad to London,February 2023,4.0,5.0,2.0,4.0,5.0,yes,✅ Trip Verified | This was my first time flyin...,3.0,NaN
4,2023-02-15,1,833131,NaN,Couple Leisure,Economy Class,Edinburgh to London,February 2023,3.0,2.0,NaN,1.0,1.0,no,✅ Trip Verified | Lots of cancellations and d...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,2012-08-29,4,244109,NaN,NaN,Economy Class,NaN,NaN,2.0,3.0,1.0,NaN,3.0,no,LHR-JFK-LAX-LHR. Check in was ok apart from be...,4.0,NaN
3472,2012-08-28,9,243823,NaN,NaN,Business Class,NaN,NaN,4.0,5.0,4.0,NaN,3.0,yes,LHR to HAM. Purser addresses all club passenge...,NaN,NaN
3473,2011-10-12,5,243776,NaN,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,NaN,4.0,yes,My son who had worked for British Airways urge...,NaN,NaN
3474,2011-10-11,4,243824,NaN,NaN,Premium Economy,NaN,NaN,1.0,3.0,5.0,NaN,1.0,no,London City-New York JFK via Shannon on A318 b...,NaN,NaN


In [ ]:
df.to_csv('British_Airways.csv')
!cp British_Airways.csv "drive/My Drive/"